## Video analog gauge reader with API app

sample notebook for live detecton of analog gauge with API app (docker)

In [ ]:
import cv2
import numpy as np
import time
import io
import json
import requests

from gauge_reader import detectCircle, detectLine, get_current_value
from gauge_utils import drawImage, cropImage

### Run docker app

```
docker run --name gaugereader -e "ROI=196,98,467,360" -e "MINMAXMETER=40,310,0,100" -e "MINLINELENGTH=50" -p 8080:8080 -d gaugereader
```

In [ ]:
def gauge_reader(frame):
    url = "http://localhost:8080/score"

    headers = {"Content-Type": "image/jpeg" }

    try:
        r = requests.post(url, headers=headers, data=frame)
        j = json.loads(r.content.decode("utf-8"))
        return j
    except:
        return json.loads("{\"value\": 0.0}")

In [ ]:
%%time
# test docker app
gauge_img = './captured.jpg'
url = "http://localhost:8080/score"
headers = {"Content-Type": "image/jpeg" }

with open(gauge_img, 'rb') as roi:
    r = requests.post(url, headers=headers, data=roi)
    
    pred = json.loads(r.content.decode("utf-8"))

print(pred["value"])

### Live detection with API

In [ ]:
### capture camera
#cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture("rtsp://192.168.50.72")
cap = cv2.VideoCapture("./sample/sample.avi")

# img resolution 640x480
#print('width: {0}, height: {1}'.format(cap.get(3),cap.get(4)))

prev_time = 0
FPS = 2
fbuff = None

while(True):
    ret, frame = cap.read()
    curr_time = time.time() - prev_time
    if (ret) and (curr_time > 1./FPS):
        prev_time = time.time()
                     
        # encode img to jpg format
        is_success, buffer = cv2.imencode(".jpg", frame)
        io_buf = io.BytesIO(buffer)
        pred = gauge_reader(io_buf)

        # ignore undetected value
        if (pred["value"] > 0):
            cv2.line(frame, (pred["line"]["x1"], pred["line"]["y1"]), (pred["line"]["x2"], pred["line"]["y2"]),(0, 255, 0), 2)            
            cv2.putText(frame, "Mem %.0f" % pred["value"], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            
        cv2.imshow('frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    elif not ret:
        break

    # comment this if you're using live stream source
    time.sleep(0.1)
    
cap.release()
cv2.destroyAllWindows()
